# Poisson 2D

This notebook solves the linear problem:

$$ - \Delta u = f \text{ in } \Omega = [0,1]^2$$ 

with

$$ f(x, y) = 10 \exp^{-50 [(x - 1/2)^2 + (y - 1/2)^2)]} $$

and boundary conditions given by

$$    u(x, y) = 0 \text{ for } x = 0 \text{ or } x = 1 $$
and

$$ \frac{\partial u}{\partial n}(x, y) = sin(5 x) \text{ for } y = 0 \text{ or } y = 1. $$

The code setting up the variational problem is taken almost verbatim from the original dolfin demo:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as pl
import numpy as np
from IPython.display import Math

from dolfin import *

parameters["form_compiler"]["cpp_optimize"] = False
parameters["form_compiler"]["optimize"] = False

mesh = UnitSquareMesh(64, 64)

def poisson(V):
    """ Solves Poisson's eq on the unit square.
    This code is based on DOLFIN's homonymous demo, (C) 2007-2011 Anders Logg

        V: FunctionSpace.
    """
    
    # Define Dirichlet boundary (x = 0 or x = 1)
    def boundary(x):
        return x[0] < DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS

    # Define boundary condition
    # Need to project to work around lack of evaluate_dofs() for Hermite
    u0 = project(Constant(0.0), V)
    bc = DirichletBC(V, u0, boundary)

    # Define variational problem
    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("10*exp(-50*(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2)))", degree=2)
    g = Expression("sin(5*x[0])", degree=2)
    a = inner(grad(u), grad(v))*dx
    L = f*v*dx + g*v*ds

    # Compute solution
    u = Function(V)
    solve(a == L, u, bc)
    return u

## Four different solutions

We use P1, P2 and P3 Lagrange elements as well as P3 Hermite elements and compare the results.

In [ ]:
W = [None] + [FunctionSpace(mesh, "Lagrange", i) for i in range(1, 4)]
lag = [None] + [poisson(W[p]) for p in range(1, 4)]

pl.figure(figsize=(10, 4))
for i in range(1, 4):
    pl.subplot(1, 3, i)
    plot(lag[i], title = "P%d Lagrange" % i, cmap='bone')

In [ ]:
V = FunctionSpace(mesh, "Hermite", 3)
her = poisson(V)
plot(her, cmap='bone')
_ = pl.title("P3 Hermite")

## Relative errors

We find a consistent relative error of around 16% measured in the $H^1$ norm and 4% in the $L^2$ norm.

In [ ]:
pl.figure(figsize=(12,4))
eqs = ''#\\begin{align}'
for i in range(1,4):
    diff = project(lag[i] - her, V)
    diff.vector().abs()
    eqs += '\\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{L_2}}{||u_{\\text{l%d}}||_{L_2}} \\approx %.4f,\ \ \ '\
           ' \\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{H_1}}{||u_{\\text{l%d}}||_{H_1}} \\approx %.4f \\\~ '\
           % (i, i, norm(diff, 'L2') / norm(lag[i], 'L2'), i, i, norm(diff, 'H1') / norm(lag[i], 'H1'))
    pl.subplot(1,3,i)
    plot(diff, title="|Lag%d - Her3|" % i, cmap='bone')
eqs += ''#'\\end{align}'
Math(eqs)

A final plot with some "cross-sections" at fixed ordinates for the P3 Lagrange vs Hermite elements. We need to cap the relative error because of very small denominators.

In [ ]:
xx = np.linspace(0,1,100)
pl.figure(figsize=(8,6))
error_threshold = 0.5

for y in np.linspace(0, 1, 4):
    pl.plot(xx, [min(error_threshold, np.abs(diff(x,y)/lag[3](x,y))) for x in xx],
            label="y = %.1f" % y)
    #pl.plot(xx, [diff(x,y) for x in xx], label="D@%.1f" % y)
    #pl.plot(xx, [her([x,y]) for x in xx], label="H@%.1f" % y)
    #pl.plot(xx, [lag([x,y]) for x in xx], label="L@%.1f" % y)
pl.title("Relative error (capped at %.1f) at multiple ordinates" % error_threshold)
_ = pl.legend()